In [1]:
import moviepy.editor as mp
import cv2
import os

# Функция для извлечения аудио из видео
def extract_audio_from_video(video_path, output_audio_path):
    video = mp.VideoFileClip(video_path)
    video.audio.write_audiofile(output_audio_path)

# Функция для нарезки видео на фреймы
def extract_frames_from_video(video_path, output_frames_dir, frame_rate=1):
    # Создаём директорию для сохранения фреймов, если её нет
    os.makedirs(output_frames_dir, exist_ok=True)
    
    # Открываем видео
    vidcap = cv2.VideoCapture(video_path)
    success, image = vidcap.read()
    
    count = 0
    while success:
        # Сохраняем каждый кадр через указанное количество фреймов (например, 1 кадр в секунду)
        if count % frame_rate == 0:
            frame_filename = os.path.join(output_frames_dir, f"frame_{count}.jpg")
            cv2.imwrite(frame_filename, image)  # Сохраняем кадр
            print(f'Saved frame: {frame_filename}')
        
        success, image = vidcap.read()  # Читаем следующий кадр
        count += 1
    
    vidcap.release()

# Путь к вашему видеофайлу MP4
video_path = "./videos/video10.mp4"

# Путь для сохранения аудио в формате WAV
output_audio_path = "./audio_files/test_audio.wav"

# Директория для сохранения фреймов
output_frames_dir = "./frames/test_video"

# Извлечение аудио из видео
extract_audio_from_video(video_path, output_audio_path)

# Нарезка видео на фреймы (1 кадр в секунду)
extract_frames_from_video(video_path, output_frames_dir, frame_rate=1)  # frame_rate = 30 означает каждый 30-й кадр


MoviePy - Writing audio in ./audio_files/test_audio.wav


MoviePy - Done.


Saved frame: ./frames/test_video\frame_0.jpg
Saved frame: ./frames/test_video\frame_1.jpg
Saved frame: ./frames/test_video\frame_2.jpg
Saved frame: ./frames/test_video\frame_3.jpg
Saved frame: ./frames/test_video\frame_4.jpg
Saved frame: ./frames/test_video\frame_5.jpg
Saved frame: ./frames/test_video\frame_6.jpg
Saved frame: ./frames/test_video\frame_7.jpg
Saved frame: ./frames/test_video\frame_8.jpg
Saved frame: ./frames/test_video\frame_9.jpg
Saved frame: ./frames/test_video\frame_10.jpg
Saved frame: ./frames/test_video\frame_11.jpg
Saved frame: ./frames/test_video\frame_12.jpg
Saved frame: ./frames/test_video\frame_13.jpg
Saved frame: ./frames/test_video\frame_14.jpg
Saved frame: ./frames/test_video\frame_15.jpg
Saved frame: ./frames/test_video\frame_16.jpg
Saved frame: ./frames/test_video\frame_17.jpg
Saved frame: ./frames/test_video\frame_18.jpg
Saved frame: ./frames/test_video\frame_19.jpg
Saved frame: ./frames/test_video\frame_20.jpg
Saved frame: ./frames/test_video\frame_21.jp

In [2]:
import torch
from transformers import Wav2Vec2ForCTC, Wav2Vec2Processor

def load_model_and_processor(model_name):
    # Загрузка процессора и модели
    processor = Wav2Vec2Processor.from_pretrained(model_name)
    model = Wav2Vec2ForCTC.from_pretrained(model_name)
    return processor, model

# Пример загрузки модели
model_name = "jonatasgrosman/wav2vec2-large-xlsr-53-russian"
processor, model = load_model_and_processor(model_name)


In [13]:
import nemo.collections.asr as nemo_asr
asr_model = nemo_asr.models.ASRModel.from_pretrained("nvidia/stt_ru_conformer_transducer_large")

transcriptions = asr_model.transcribe(["file.wav"])

ModuleNotFoundError: No module named 'nemo'

In [12]:
import torchaudio
from pydub import AudioSegment

# Функция для загрузки аудио и конвертации в формат, пригодный для модели
def load_audio(file_path, target_sample_rate=16000):
    # Используем pydub для преобразования аудио в моно и нужную частоту дискретизации
    audio = AudioSegment.from_file(file_path)
    audio = audio.set_frame_rate(target_sample_rate).set_channels(1)
    
    # Преобразуем аудио в массив numpy
    audio_array = torch.tensor(audio.get_array_of_samples(), dtype=torch.float32) / 2 ** 16 / 2  # нормализация
    return audio_array

# Загрузка аудиофайла
audio_path = "./audio_files/test_audio.wav"  # Замените на ваш путь к файлу
audio_input = load_audio(audio_path)

# Преобразуем аудио в тензор нужной формы с указанием частоты дискретизации
input_values = processor(audio_input, return_tensors="pt", padding="longest", sampling_rate=16000).input_values

# Модель предсказывает логиты, которые затем преобразуются в текст
with torch.no_grad():
    logits = model(input_values).logits

# Извлекаем предсказания
predicted_ids = torch.argmax(logits, dim=-1)

# Преобразуем предсказания в текст
transcription = processor.decode(predicted_ids[0])

print("Transcription:", transcription)


ValueError: The model corresponding to this feature extractor: Wav2Vec2FeatureExtractor {
  "do_normalize": true,
  "feature_extractor_type": "Wav2Vec2FeatureExtractor",
  "feature_size": 1,
  "padding_side": "right",
  "padding_value": 0.0,
  "processor_class": "Wav2Vec2ProcessorWithLM",
  "return_attention_mask": true,
  "sampling_rate": 16000
}
 was trained using a sampling rate of 16000. Please make sure that the provided `raw_speech` input was sampled with 16000 and not 20000.